In [1]:
import os
import pandas as pd

# Yol tanımları
BASE_DIR = "D:/SYSTEM/DataGast/labeled-images"  # Görsellerin olduğu klasör
CSV_FILE = "D:/SYSTEM/DataGast/labeled-images/image-labels.csv"

# CSV dosyasını yükle
df = pd.read_csv(CSV_FILE)

# Görsellerin varlığını kontrol et
missing_files = []
for _, row in df.iterrows():
    file_name = row['Video file']  # Dosya adı sütunu
    file_path = os.path.join(BASE_DIR, file_name)
    if not os.path.exists(file_path):
        missing_files.append(file_path)

# Eksik dosyaları raporla
if missing_files:
    print("Eksik dosyalar bulundu:")
    for file in missing_files:
        print(file)
else:
    print("Tüm dosyalar mevcut!")


Eksik dosyalar bulundu:
D:/SYSTEM/DataGast/labeled-images\000e3fee-7f5c-4819-9f9c-4c983b68888a
D:/SYSTEM/DataGast/labeled-images\001a41c2-2a5d-40b1-8fd5-b5f2f292277b
D:/SYSTEM/DataGast/labeled-images\006af0aa-2044-4477-964d-10d9e043fb78
D:/SYSTEM/DataGast/labeled-images\00832522-ab8e-4b98-bfce-93a777929571
D:/SYSTEM/DataGast/labeled-images\012ab888-64e6-4361-9745-f52b4a03ba75
D:/SYSTEM/DataGast/labeled-images\014a6880-6306-47ab-8766-adc17507b7c3
D:/SYSTEM/DataGast/labeled-images\019975d6-14e2-4a71-9e35-934f2280fab6
D:/SYSTEM/DataGast/labeled-images\01b2ec26-e814-4053-a820-37c9b703d47a
D:/SYSTEM/DataGast/labeled-images\020945d2-ade3-4cf9-b15d-634729b2b4a2
D:/SYSTEM/DataGast/labeled-images\0225440c-73b0-4c5e-b4b4-5bfe306834c9
D:/SYSTEM/DataGast/labeled-images\022ade3a-d78e-49b6-a447-609bdc8da113
D:/SYSTEM/DataGast/labeled-images\02387cd3-612f-4358-a4d8-4f3116fb3db0
D:/SYSTEM/DataGast/labeled-images\026d749a-7017-4a08-8920-8ed44a168f7e
D:/SYSTEM/DataGast/labeled-images\02952c26-912d-447b-

In [2]:
import os
import pandas as pd

# CSV Dosyasını Yükleme
csv_path = "D:/SYSTEM/DataGast/labeled-images/image-labels.csv"
df = pd.read_csv(csv_path)

# Dosya Uzantılarını Kontrol Etme ve Ekleme
image_dir = "D:/SYSTEM/DataGast/labeled-images"
valid_extensions = ['.jpg', '.jpeg', '.png']

updated_files = []
for file in df['Video file']:
    found = False
    for ext in valid_extensions:
        if os.path.exists(os.path.join(image_dir, file + ext)):
            updated_files.append(file + ext)
            found = True
            break
    if not found:
        updated_files.append(file)  # Eğer dosya bulunamazsa eski adı ekle

# Yeni CSV Oluşturma
df['Video file'] = updated_files
df.to_csv("D:/SYSTEM/DataGast/labeled-images/image-labels-updated.csv", index=False)
print("Dosya uzantıları eklendi, yeni CSV oluşturuldu!")


Dosya uzantıları eklendi, yeni CSV oluşturuldu!


In [13]:
# Eksik dosyaları CSV'den kaldır
import os
import pandas as pd

csv_path = "D:/SYSTEM/DataGast/labeled-images/image-labels-updated.csv"
image_dir = "D:/SYSTEM/DataGast/labeled-images"

df = pd.read_csv(csv_path)

# Eksik dosyaları bul
existing_files = []
for file in df['Video file']:
    if os.path.exists(os.path.join(image_dir, file)):
        existing_files.append(True)
    else:
        existing_files.append(False)

# Sadece var olan dosyaları tut
df = df[existing_files]
df.to_csv("D:/SYSTEM/DataGast/labeled-images/image-labels-cleaned.csv", index=False)
print("Eksik dosyalar temizlendi, yeni CSV oluşturuldu!")


Eksik dosyalar temizlendi, yeni CSV oluşturuldu!


In [3]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Klasör tanımları
BASE_DIR = "D:/SYSTEM/DataGast/labeled-images"  # Ana klasör yolu
CSV_FILE = "D:/SYSTEM/DataGast/labeled-images/image-labels-updated.csv"  # Güncellenmiş CSV
OUTPUT_DIR = "D:/SYSTEM/DataGast/prepared-dataset"  # Çıkış klasörü

# Train, Validation, Test klasörlerini oluştur
TRAIN_DIR = os.path.join(OUTPUT_DIR, "train")
VAL_DIR = os.path.join(OUTPUT_DIR, "validation")
TEST_DIR = os.path.join(OUTPUT_DIR, "test")

for folder in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    os.makedirs(folder, exist_ok=True)

# CSV dosyasını yükle
df = pd.read_csv(CSV_FILE)

# Görselleri bulmak için tüm alt klasörleri tarama
existing_files = []
for _, row in df.iterrows():
    image_name = row['Video file']  # Görsel isimlerinin bulunduğu sütun
    for root, _, files in os.walk(BASE_DIR):
        for file in files:
            if file.startswith(image_name):  # Dosya ismi eşleşiyorsa
                file_path = os.path.join(root, file)
                existing_files.append((file_path, row['Finding']))  # Sınıf etiketiyle ekle
                break

# Eğitim, doğrulama ve test setlerine bölme
train_files, temp_files = train_test_split(existing_files, test_size=0.3, random_state=42, stratify=[x[1] for x in existing_files])
val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42, stratify=[x[1] for x in temp_files])

# Görselleri taşıma fonksiyonu
def move_files(file_list, dest_dir):
    for file_path, label in file_list:
        label_dir = os.path.join(dest_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(file_path, os.path.join(label_dir, os.path.basename(file_path)))

# Dosyaları taşı
move_files(train_files, TRAIN_DIR)
move_files(val_files, VAL_DIR)
move_files(test_files, TEST_DIR)

print("Görseller başarıyla train, validation ve test klasörlerine taşındı!")

Görseller başarıyla train, validation ve test klasörlerine taşındı!
